# Step 2) Preparing weather data

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from tsm.weather import interpolate_weather_data, add_ewm_lags

In [14]:
from tsm.data_utils import compress_memory_usage

## Process Train

In [19]:
weather_file = "data/comp/weather_train.pkl"

In [20]:
weather_data, _ = compress_memory_usage(add_ewm_lags(interpolate_weather_data(pd.read_pickle(weather_file))), {})

Splitter will return list of 16 dataframe
Splitter will return list of 16 dataframe



Memory usage pre-compression was 37.45671558380127
Memory usage after-compression was 20.927773475646973
This is  55.87188612099644% of the initial size


In [21]:
weather_data.to_pickle(weather_file)

## Process Test

In [19]:
weather_file = "data/comp/weather_test.pkl"

In [20]:
weather_data, _ = compress_memory_usage(add_ewm_lags(interpolate_weather_data(pd.read_pickle(weather_file))), {})

Splitter will return list of 16 dataframe
Splitter will return list of 16 dataframe



Memory usage pre-compression was 37.45671558380127
Memory usage after-compression was 20.927773475646973
This is  55.87188612099644% of the initial size


In [21]:
weather_data.to_pickle(weather_file)